# Extraction des heures à partir des fichiers textes dans Dropbox

Mon script de suivi des heures entre les nouvelles entrées comme des fichiers textes sur Dropbox. Je veux les synthétiser en un seul tableur, avec les informations pertinentes, pour faciliter l'entrée dans la feuille de temps ensuite.

In [3]:
%pip install pandas

     |████████████████████████████████| 10.7 MB 732 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
from pathlib import Path

Les fichiers sont stockés dans un sous-dossier Dropbox quand tout va bien, et parfois par erreur dans le dossier racine. On pourra s'occuper des fichiers dans le sous-dossier pour commencer.

In [2]:
racine = Path.home() / 'Dropbox/'
boite_de_dépôt = racine / 'Boite de dépôt'

fichiers_correctement_déposés = list(boite_de_dépôt.glob('*.txt'))
fichiers_incorrectement_déposés = list(racine.glob('*.txt'))

In [16]:
racine = Path.home() / 'Library' / 'Mobile Documents' / 'iCloud~is~workflow~my~workflows' / 'Documents'
boite_de_dépôt = racine / 'Journal de bord/' 

fichiers_correctement_déposés = [f for f in boite_de_dépôt.glob('*.txt') if 'complétée' in str(f)]
#fichiers_incorrectement_déposés = list(racine.glob('*.txt'))

Un fichier typique ressemble à ceci:

In [17]:
with fichiers_correctement_déposés[0].open() as f:
    print(str(fichiers_correctement_déposés[0]))
    print(f.read())

/Users/emilejetzer/Library/Mobile Documents/iCloud~is~workflow~my~workflows/Documents/Journal de bord/2021-03-30T09_41_49-04_00 Tâche complétée.txt
Temps (min): 30
Nbr d'heures : 0,5
Payeur: Daq pour projet 2
Demandeur: 
Description des travaux effectués: ""
Atelier: 0
Début: 30 mars 2021 09:11
Fin: 30 mars 2021 09:41
Lieu: H3T 0A1


On peut les lire ligne par ligne pour produire une liste:

In [4]:
with fichiers_correctement_déposés[0].open() as f:
    nouvelle_entrée = [0, '', False]
    
    for ligne in f.readlines():
        champ, valeur = ligne.strip().split(':', 1)
        if champ.startswith('Temps'):
            # Certains fichiers ont le temps en heures, d'autres en minutes
            champ = champ.strip(')').split()
            if len(champ) > 1:
                unité = champ[1].strip('(')
            else:
                unité = 'min'
            
            valeur = float(valeur.strip())
            if unité == 'h':
                valeur *= 60
            
            nouvelle_entrée[0] = valeur
        elif champ.startswith('Tâche'):
            valeur = valeur.strip(' "')
            nouvelle_entrée[1] = valeur
        elif champ.startswith('Atelier'):
            nouvelle_entrée[2] = '1' in valeur

print(nouvelle_entrée)

[5.0, 'Test de l’ajout de photos', False]


On utilise la date & heure de création du fichier (heureusement préservée par Dropbox) pour obtenir le moment d'entrée.

In [20]:
étampe = fichiers_correctement_déposés[0].stat().st_birthtime

In [21]:
from datetime import datetime as Dt
Dt.fromtimestamp(étampe)

datetime.datetime(2021, 3, 30, 9, 42, 2)

On peut l'ajouter à la liste:

In [7]:
with fichiers_correctement_déposés[0].open() as f:
    nouvelle_entrée = [None, 0, '', False]
    
    étampe = Dt.fromtimestamp(fichiers_correctement_déposés[0].stat().st_birthtime)
    nouvelle_entrée[0] = étampe
    
    for ligne in f.readlines():
        champ, valeur = ligne.strip().split(':', 1)
        if champ.startswith('Temps'):
            # Certains fichiers ont le temps en heures, d'autres en minutes
            champ = champ.strip(')').split()
            if len(champ) > 1:
                unité = champ[1].strip('(')
            else:
                unité = 'min'
            
            valeur = float(valeur.strip())
            if unité == 'h':
                valeur *= 60
            
            nouvelle_entrée[1] = valeur
        elif champ.startswith('Tâche'):
            valeur = valeur.strip(' "')
            nouvelle_entrée[2] = valeur
        elif champ.startswith('Atelier'):
            nouvelle_entrée[3] = '1' in valeur

print(nouvelle_entrée)

[datetime.datetime(2021, 1, 28, 13, 22, 45), 5.0, 'Test de l’ajout de photos', False]


Pour que ce soit vraiment utile, il faut passer à travers toute la liste de fichiers, et garder chaque entrée dans une liste.

In [22]:
entrées = []

for chemin in fichiers_correctement_déposés:
    nouvelle_entrée = [None, 0, '', False]
    
    with chemin.open() as f:
        étampe = Dt.fromtimestamp(chemin.stat().st_birthtime)
        nouvelle_entrée[0] = étampe

        for ligne in f.readlines():
            champ, valeur = ligne.strip().split(':', 1)
            if champ.startswith('Temps'):
                # Certains fichiers ont le temps en heures, d'autres en minutes
                champ = champ.strip(')').split()
                if len(champ) > 1:
                    unité = champ[1].strip('(')
                else:
                    unité = 'min'

                valeur = float(valeur.replace(',', '.').strip())
                if unité == 'h':
                    valeur *= 60

                nouvelle_entrée[1] = valeur
            elif champ.startswith('Tâche'):
                valeur = valeur.strip(' "')
                nouvelle_entrée[2] = valeur
            elif champ.startswith('Atelier'):
                nouvelle_entrée[3] = '1' in valeur
    
    entrées.append(nouvelle_entrée)

Pour faciliter la manipulation des données, on les transforme en cadre Pandas:

In [23]:
données = pd.DataFrame(entrées, columns=['Date', 'Temps (min)', 'Description', 'Atelier'])

In [26]:
données.to_excel(str(boite_de_dépôt / 'résumé.xls'))

<ipython-input-26-7fb553687c3e>:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  données.to_excel(str(boite_de_dépôt / 'résumé.xls'))


In [25]:
%pip install xlwt

     |████████████████████████████████| 99 kB 3.2 MB/s eta 0:00:011
Note: you may need to restart the kernel to use updated packages.


On répète pour les fichiers qui n'étaient pas à la bonne place:

In [11]:
entrées = []

for chemin in fichiers_incorrectement_déposés:
    nouvelle_entrée = [None, 0, '', False]
    
    with chemin.open() as f:
        étampe = Dt.fromtimestamp(chemin.stat().st_birthtime)
        nouvelle_entrée[0] = étampe

        for ligne in f.readlines():
            champ, valeur = ligne.strip().split(':', 1)
            if champ.startswith('Temps'):
                # Certains fichiers ont le temps en heures, d'autres en minutes
                champ = champ.strip(')').split()
                if len(champ) > 1:
                    unité = champ[1].strip('(')
                else:
                    unité = 'min'

                valeur = float(valeur.replace(',', '.').strip())
                if unité == 'h':
                    valeur *= 60

                nouvelle_entrée[1] = valeur
            elif champ.startswith('Tâche'):
                valeur = valeur.strip(' "')
                nouvelle_entrée[2] = valeur
            elif champ.startswith('Atelier'):
                nouvelle_entrée[3] = '1' in valeur
    
    entrées.append(nouvelle_entrée)

données = pd.DataFrame(entrées, columns=['Date', 'Temps (min)', 'Description', 'Atelier'])
données.to_excel(str(boite_de_dépôt / 'résumé2.xls'))